geemap에서 참조하는 라이브러리가 굉장히 많으므로 충돌로 인한 이상 혹은 오류 발생이 잦을 수 있음

In [ ]:
# !pip install geemap

In [ ]:
# !pip install ipympl

In [ ]:
# !pip install ipywidgets

## Import libraries

In [1]:
import ee
import geemap

In [5]:
%matplotlib widget

In [2]:
geemap.Map()

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# ee.Authenticate()
# ee.Initialize()

In [2]:
geemap.ee_initialize()

## The map function

In [7]:
myList = ee.List.sequence(1, 10)
print(myList.getInfo())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [8]:
def computeSquares(number):
    return ee.Number(number).pow(2)


squares = myList.map(computeSquares)
print(squares.getInfo())

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


In [9]:
squares = myList.map(lambda number: ee.Number(number).pow(2))
print(squares.getInfo())

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


## Creating cloud-free composites

In [10]:
Map = geemap.Map()
fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    ee.Filter.eq('country_na', 'Netherlands')
)

Map.addLayer(fc, {'color': 'ff000000'}, "Netherlands")
Map.centerObject(fc)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [11]:
years = ee.List.sequence(2013, 2021)

In [12]:
def yearly_image(year):

    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = start_date.advance(1, "year")

    collection = (
        ee.ImageCollection('LANDSAT/LC08/C01/T1')
        .filterDate(start_date, end_date)
        .filterBounds(fc)
    )

    image = ee.Algorithms.Landsat.simpleComposite(collection).clipToCollection(fc)

    return image

In [13]:
images = years.map(yearly_image)

In [14]:
vis_params = {'bands': ['B5', 'B4', 'B3'], 'max': 128}
for index in range(0, 9):
    image = ee.Image(images.get(index))
    layer_name = "Year " + str(index + 2013)
    Map.addLayer(image, vis_params, layer_name)

## Creating timeseries

In [15]:
collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED").filterMetadata(
    'CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10
)

In [16]:
start_date = '2016-01-01'
end_date = '2022-12-31'
region = ee.Geometry.BBox(-122.5549, 37.6968, -122.3446, 37.8111)

In [17]:
images = geemap.create_timeseries(
    collection, start_date, end_date, region, frequency='year', reducer='median'
)
images.size().getInfo()

7

In [18]:
Map = geemap.Map()

vis_params = {"min": 0, "max": 4000, "bands": ["B8", "B4", "B3"]}
labels = [str(y) for y in range(2016, 2023)]

Map.addLayer(images, vis_params, "Sentinel-2", False)
Map.add_time_slider(images, vis_params, time_interval=2, labels=labels)
Map.centerObject(region)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## NAIP timelapse

In [3]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [4]:
roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-99.1019, 47.1274, -99.0334, 47.1562)
    Map.addLayer(roi)
    Map.centerObject(roi)

In [5]:
collection = geemap.naip_timeseries(roi, start_year=2009, end_year=2022, RGBN=True)

In [6]:
years = geemap.image_dates(collection, date_format='YYYY').getInfo()
print(years)

['2009', '2010', '2012', '2014', '2015', '2016', '2017', '2018', '2019', '2020']


In [7]:
size = len(years)
images = collection.toList(size)
for i in range(size):
    image = ee.Image(images.get(i))
    Map.addLayer(image, {'bands': ['N', 'R', 'G']}, years[i])
Map

Map(bottom=368428.0, center=[47.15984001304432, -99.09667968750001], controls=(WidgetControl(options=['positio…

In [8]:
timelapse = geemap.naip_timelapse(
    roi,
    out_gif="naip.gif",
    bands=['N', 'R', 'G'],
    frames_per_second=3,
    title='NAIP Timelapse',
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\user\Desktop\산림과학원\221129 GEE-Python API\naip.gif
ffmpeg is not installed on your computer.


Output()

## Landsat timelapse

In [ ]:
Map = geemap.Map()
Map

In [ ]:
roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-74.7222, -8.5867, -74.1596, -8.2824)
    Map.addLayer(roi)
    Map.centerObject(roi)

In [ ]:
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif='landsat.gif',
    start_year=1984,
    end_year=2022,
    start_date='01-01',
    end_date='12-31',
    bands=['SWIR1', 'NIR', 'Red'],
    frames_per_second=5,
    title='Landsat Timelapse',
    progress_bar_color='blue',
    mp4=True,
)
geemap.show_image(timelapse)

![](https://i.imgur.com/VL1V1Y4.gif)

In [ ]:
Map = geemap.Map()
roi = ee.Geometry.BBox(-115.5541, 35.8044, -113.9035, 36.5581)
Map.addLayer(roi)
Map.centerObject(roi)
Map

In [ ]:
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif='las_vegas.gif',
    start_year=1984,
    end_year=2022,
    bands=['NIR', 'Red', 'Green'],
    frames_per_second=5,
    title='Las Vegas, NV',
    font_color='blue',
)
geemap.show_image(timelapse)

![](https://i.imgur.com/LzWyyZW.gif)

In [ ]:
Map = geemap.Map()
roi = ee.Geometry.BBox(113.8252, 22.1988, 114.0851, 22.3497)
Map.addLayer(roi)
Map.centerObject(roi)
Map

In [ ]:
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif='hong_kong.gif',
    start_year=1990,
    end_year=2022,
    start_date='01-01',
    end_date='12-31',
    bands=['SWIR1', 'NIR', 'Red'],
    frames_per_second=3,
    title='Hong Kong',
)
geemap.show_image(timelapse)

![](https://i.imgur.com/0nLRj22.gif)

## Sentinel-1 timelapse

In [9]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [10]:
roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(117.1132, 3.5227, 117.2214, 3.5843)
    Map.addLayer(roi)
    Map.centerObject(roi)

In [11]:
timelapse = geemap.sentinel1_timelapse(
    roi,
    out_gif='sentinel1.gif',
    start_year=2019,
    end_year=2019,
    start_date='04-01',
    end_date='08-01',
    frequency='day',
    vis_params={"min": -30, "max": 0},
    palette="Greys",
    frames_per_second=3,
    title='Sentinel-1 Timelapse',
    add_colorbar=True,
    colorbar_bg_color='gray',
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\user\Desktop\산림과학원\221129 GEE-Python API\sentinel1.gif
ffmpeg is not installed on your computer.


Output()

## Sentinel-2 timelapse

In [ ]:
Map = geemap.Map()

In [ ]:
roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-74.7222, -8.5867, -74.1596, -8.2824)
    Map.addLayer(roi)
    Map.centerObject(roi)

In [ ]:
timelapse = geemap.sentinel2_timelapse(
    roi,
    out_gif='sentinel2.gif',
    start_year=2016,
    end_year=2021,
    start_date='01-01',
    end_date='12-31',
    frequency='year',
    bands=['SWIR1', 'NIR', 'Red'],
    frames_per_second=3,
    title='Sentinel-2 Timelapse',
)
geemap.show_image(timelapse)

![](https://i.imgur.com/BmQdo9j.gif)

## Visualizing Global Land Cover Data Products

### Creating Dynamic World Land Cover Composites

- App: https://www.dynamicworld.app
- App2: https://earthoutreach.users.earthengine.app/view/dynamicworld
- Paper: https://doi.org/10.1038/s41597-022-01307-4
- Model: https://github.com/google/dynamicworld
- Training data: https://doi.pangaea.de/10.1594/PANGAEA.933475
- Data: https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1
- JavaScript tutorial: https://developers.google.com/earth-engine/tutorials/community/introduction-to-dynamic-world-pt-1

In [12]:
Map = geemap.Map()
Map.add_basemap('HYBRID')

In [14]:
# Set the region of interest by simply drawing a polygon on the map
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-89.7088, 42.9006, -89.0647, 43.2167)
    Map.addLayer(region, {}, 'Region')

Map.centerObject(region)

In [15]:
# Set the date range
start_date = '2021-01-01'
end_date = '2022-01-01'

In [16]:
# Create a Sentinel-2 image composite
image = geemap.dynamic_world_s2(region, start_date, end_date, clip=True)
vis_params = {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 3000}
Map.addLayer(image, vis_params, 'Sentinel-2 image')

In [17]:
# Create Dynamic World land cover composite
landcover = geemap.dynamic_world(
    region, start_date, end_date, clip=True, return_type='class'
)

dwVisParams = {
    "min": 0,
    "max": 8,
    "palette": [
        "#419BDF",
        "#397D49",
        "#88B053",
        "#7A87C6",
        "#E49635",
        "#DFC35A",
        "#C4281B",
        "#A59B8F",
        "#B39FE1",
    ],
}

Map.addLayer(landcover, dwVisParams, 'Land Cover Class')
Map

Map(center=[43.05865000086152, -89.38675], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [18]:
Map.add_legend(title='Dynamic World Land Cover', builtin_legend='Dynamic_World')

In [19]:
landcover = geemap.dynamic_world(
    region, start_date, end_date, clip=True, return_type='visualize'
)
Map.addLayer(landcover, {}, 'Land Cover Visualize')

In [20]:
landcover = geemap.dynamic_world(
    region, start_date, end_date, clip=True, return_type='probability'
)
Map.addLayer(landcover, {}, 'Land Cover Probability')

In [21]:
# Create Dynamic World land cover composite
landcover = geemap.dynamic_world(
    region, start_date, end_date, clip=True, return_type='hillshade'
)
Map.addLayer(landcover, {}, 'Land Cover')

![](https://i.imgur.com/GEzsSii.png)

---


In [109]:
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands([ndvi])

In [122]:
def maskS2coluds(image):
    qa = image.select('QA60');
    
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    
    mask = (qa.bitwiseAnd(cloudBitMask).eq(0) and (qa.bitwiseAnd(cirrusBitMask).eq(0)))
        
    return image.updateMask(mask)

In [123]:
start_date = '2022-01-01'
end_date = '2022-02-01'
mnt = [126.964207, 37.60158914]
region = ee.Geometry.Point(mnt)

In [153]:
Map = geemap.Map()

collection = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
       .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
       .filterDate(start_date, end_date)
       .filterBounds(region)
       .map(addNDVI)
       .map(maskS2coluds)
)

vis = {
    'min':0.0,
    'max':3000,
    'bands':['B4', 'B3', 'B2'],
}

In [164]:
print(collection)

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.map",
    "arguments": {
      "baseAlgorithm": {
        "functionDefinitionValue": {
          "argumentNames": [
            "_MAPPING_VAR_0_0"
          ],
          "body": {
            "functionInvocationValue": {
              "functionName": "Image.updateMask",
              "arguments": {
                "image": {
                  "argumentReference": "_MAPPING_VAR_0_0"
                },
                "mask": {
                  "functionInvocationValue": {
                    "functionName": "Image.eq",
                    "arguments": {
                      "image1": {
                        "functionInvocationValue": {
                          "functionName": "Image.bitwiseAnd",
                          "arguments": {
                            "image1": {
                              "functionInvocationValue": {
                                "functionName": "Image.select",
   

In [160]:
def addDate(image):
    return (image.reduceRegions({
                    'collection': FC,
                    'reducer': ee.Reducer.median(),
                    'scale': 10})
                .copyProperties(image, ['system:time_start'])).flatten()

In [155]:
FC = ee.FeatureCollection(collection)

In [162]:
collection.map(addDate)

EEException: Unrecognized argument type to convert to a FeatureCollection: {'collection': <ee.featurecollection.FeatureCollection object at 0x000001A30F146F40>, 'reducer': <ee.Reducer object at 0x000001A30EAA9D30>, 'scale': 10}

In [163]:
ee.Image.reduceRegion?

Signature: ee.Image.reduceRegion(*args, **kwargs)
Docstring:
Apply a reducer to all the pixels in a specific region. Either the reducer
must have the same number of inputs as the input image has bands, or it
must have a single input and will be repeated for each band. Returns a
dictionary of the reducer's outputs.

Args:
  image: The image to reduce.
  reducer: The reducer to apply.
  geometry: The region over which to reduce data.  Defaults to
      the footprint of the image's first band.
  scale: A nominal scale in meters of the projection to work in.
  crs: The projection to work in. If unspecified, the projection of
      the image's first band is used. If specified in addition to
      scale, rescaled to the specified scale.
  crsTransform: The list of CRS transform values.  This is
      a row-major ordering of the 3x2 transform matrix.
      This option is mutually exclusive with 'scale', and
      replaces any transform already set on the projection.
  bestEffort: If the polyg

In [156]:
geemap.ee_to_csv(FC, filename='test.csv')

Generating URL ...
Please wait ...
Data downloaded to C:\Users\user\Desktop\산림과학원\221129 GEE-Python API\test.csv


In [141]:
params1 = collection.map(addDate).flatten()

EEException: Unrecognized argument type to convert to a FeatureCollection: {'collection': <ee.featurecollection.FeatureCollection object at 0x000001A30F485BE0>, 'reducer': <ee.Reducer object at 0x000001A30F485BB0>, 'scale': 10}

In [127]:
# 함수화해서 각 row 삽입

date=ee.Date(collection.get('system:time_start'))
print('Timestamp: ', date.format().getInfo())

Timestamp:  2022-01-07T02:27:14


In [ ]:
images = geemap.create_timeseries(
    collection, start_date, end_date, region, frequency='week', reducer='mean', date_format='YYYY-MM-dd')
images.size().getInfo()

17

In [126]:
Map.setCenter(mnt[0], mnt[1], 12)
Map.addLayer(collection, vis, 'Sentinel-2_harmonized')
Map

Map(center=[37.60158914, 126.964207], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=H…

In [40]:
Map = geemap.Map()
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

image = (collection.filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
         .filterDate('2022-01-01', '2022-02-01')
         .median())

vis = {
    'min':0.0,
    'max':3000,
    'bands':['B4', 'B3', 'B2'],
}

Map.setCenter(126.964207, 37.60158914, 12)
Map.addLayer(image, vis, 'Sentinel-2')
Map


Map(center=[37.60158914, 126.964207], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=H…

In [29]:
collection.filterMetadata?

Signature: collection.filterMetadata(name, operator, value)
Docstring:
Shortcut to add a metadata filter to a collection.

This is equivalent to self.filter(Filter().metadata(...)).

Args:
  name: Name of a property to filter.
  operator: Name of a comparison operator as defined
      by FilterCollection.  Possible values are: "equals", "less_than",
      "greater_than", "not_equals", "not_less_than", "not_greater_than",
      "starts_with", "ends_with", "not_starts_with", "not_ends_with",
      "contains", "not_contains".
  value: The value to compare against.

Returns:
  The filtered collection.
File:      c:\users\user\anaconda3\lib\site-packages\ee\collection.py
Type:      method


In [ ]:
image.getInfo()